In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
train_data = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
sell_prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
submission = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

In [4]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    train_data[col] = 0
    train_data[col] = train_data[col].astype(np.int16)

In [8]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [9]:
train_data = downcast(train_data)
sell_prices = downcast(sell_prices)
calendar = downcast(calendar)

In [10]:
df = pd.melt(frame=train_data, 
             id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
             var_name="d", value_name="sold")

In [11]:
df = pd.merge(left=df, right=calendar, how="left", on="d")
df = pd.merge(left=df, right=sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

In [12]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [13]:
print("Implementing features")
df["d"] = df["d"].apply(lambda x: int(x[2:])).astype(np.int16)
df["snap"] = df["snap_CA"] + df["snap_TX"] + df["snap_WI"]
df["snap"] = df["snap"].map(lambda x: 1 if x >= 1 else 0).astype(np.int8)
df["weekend"] = df["wday"].map(lambda x : 1 if x < 3 else 0).astype(np.int8)
df["sell_price"] = df['sell_price'].fillna(df.groupby('id')['sell_price'].transform('median'))

df = df.drop(["date", "weekday", "wm_yr_wk", "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI"], axis=1)

Implementing features


In [14]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wday,month,year,event_name_1,event_type_1,sell_price,snap,weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,8.257812,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,3.970703,0,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,2.970703,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,4.640625,0,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,2.980469,0,1


In [15]:
print("Label Encoding as below")
d_id = dict(zip(df["id"].cat.codes, df["id"]))
d_store = dict(zip(df["store_id"].cat.codes, df["store_id"]))
d_dept = dict(zip(df["dept_id"].cat.codes, df["dept_id"]))
d_cat = dict(zip(df["cat_id"].cat.codes, df["cat_id"]))
df["id"] = df["id"].cat.codes
df["item_id"] = df["item_id"].cat.codes
df["dept_id"] = df["dept_id"].cat.codes
df["cat_id"] = df["cat_id"].cat.codes
df["store_id"] = df["store_id"].cat.codes
df["state_id"] = df["state_id"].cat.codes
df["event_name_1"] = df["event_name_1"].cat.codes
df["event_type_1"] = df["event_type_1"].cat.codes

Label Encoding as below


In [16]:
# Create total sale for each store
sum_df = df.groupby(["store_id", "d"])["sold"].sum().reset_index()
df = pd.merge(left=df, right=sum_df, how="left", on=["d", "store_id"])
df.rename(columns={'sold_x': 'sold','sold_y': 'total_sale_store'}, inplace=True)

# Create total sale for each department
sum_df_dept = df.groupby(["dept_id", "d"])["sold"].sum().reset_index()
df = pd.merge(left=df, right=sum_df_dept, how="left", on=["d", "dept_id"])
df.rename(columns={'sold_x': 'sold','sold_y': 'total_sale_dept'}, inplace=True)

In [19]:
%%time
print("Calulating Lags for the days more than 28 as there could be null values in the features \
while predicting")

lags = [5,8,10,25,26,27,28,29,30,31,32,33,34,35,36,37,40,50,55,60,120]
for lag in lags:
    df['sales_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

Calulating Lags for the days more than 28 as there could be null values in the features while predicting
CPU times: user 2min 50s, sys: 47.9 s, total: 3min 38s
Wall time: 3min 39s


In [21]:
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        df[col_name] = df.groupby(['id'])['sold'].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    

Shifting period: 1
Shifting period: 7
Shifting period: 14


In [ ]:
for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    df['rolling_mean_'+str(i)] = df.groupby(['id'])['sold'].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    df['rolling_std_'+str(i)]  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

In [17]:
%%time
print("Mean Encoding")
df["state_mean"] = df.groupby("state_id")["sold"].transform("mean").astype(np.float16)
df["store_mean"] = df.groupby("store_id")["sold"].transform("mean").astype(np.float16)
df["cat_mean"] = df.groupby("cat_id")["sold"].transform("mean").astype(np.float16)
df["dept_mean"] = df.groupby("dept_id")["sold"].transform("mean").astype(np.float16)
df["state_cat_mean"] = df.groupby(["state_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["state_dept_mean"] = df.groupby(["state_id", "dept_id"])["sold"].transform("mean").astype(np.float16)
df["store_cat_mean"] = df.groupby(["store_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["store_cat_mean"] = df.groupby(["dept_id", "cat_id"])["sold"].transform("mean").astype(np.float16)
df["item_id_mean"] = df.groupby("item_id")["sold"].transform("mean").astype(np.float16)
df["item_state_mean"] = df.groupby(["item_id", "state_id"])["sold"].transform("mean").astype(np.float16)
df["item_store_mean"] = df.groupby(["item_id", "store_id"])["sold"].transform("mean").astype(np.float16)

print("Median Encoding")
df["state_median"] = df.groupby("state_id")["sold"].transform("median").astype(np.float16)
df["store_median"] = df.groupby("store_id")["sold"].transform("median").astype(np.float16)
df["cat_median"] = df.groupby("cat_id")["sold"].transform("median").astype(np.float16)
df["dept_median"] = df.groupby("dept_id")["sold"].transform("median").astype(np.float16)
df["state_cat_median"] = df.groupby(["state_id", "cat_id"])["sold"].transform("median").astype(np.float16)
df["state_dept_median"] = df.groupby(["state_id", "dept_id"])["sold"].transform("median").astype(np.float16)
df["store_cat_median"] = df.groupby(["store_id", "cat_id"])["sold"].transform("median").astype(np.float16)
df["store_cat_median"] = df.groupby(["dept_id", "cat_id"])["sold"].transform("median").astype(np.float16)
df["item_id_median"] = df.groupby("item_id")["sold"].transform("median").astype(np.float16)
df["item_state_median"] = df.groupby(["item_id", "state_id"])["sold"].transform("median").astype(np.float16)
df["item_store_median"] = df.groupby(["item_id", "store_id"])["sold"].transform("median").astype(np.float16)

print("Standard Deviation encoding")
df["state_std"] = df.groupby("state_id")["sold"].transform("std").astype(np.float16)
df["store_std"] = df.groupby("store_id")["sold"].transform("std").astype(np.float16)
df["cat_std"] = df.groupby("cat_id")["sold"].transform("std").astype(np.float16)
df["dept_std"] = df.groupby("dept_id")["sold"].transform("std").astype(np.float16)
df["state_cat_std"] = df.groupby(["state_id", "cat_id"])["sold"].transform("std").astype(np.float16)
df["state_dept_std"] = df.groupby(["state_id", "dept_id"])["sold"].transform("std").astype(np.float16)
df["store_cat_std"] = df.groupby(["store_id", "cat_id"])["sold"].transform("std").astype(np.float16)
df["store_cat_std"] = df.groupby(["dept_id", "cat_id"])["sold"].transform("std").astype(np.float16)
df["item_id_std"] = df.groupby("item_id")["sold"].transform("std").astype(np.float16)
df["item_state_std"] = df.groupby(["item_id", "state_id"])["sold"].transform("std").astype(np.float16)
df["item_store_std"] = df.groupby(["item_id", "store_id"])["sold"].transform("std").astype(np.float16)

Mean Encoding
Median Encoding
Standard Deviation encoding
CPU times: user 1min 34s, sys: 27.9 s, total: 2min 2s
Wall time: 2min 3s


In [22]:
df.to_pickle('data.pkl')
del df

NameError: name 'gc' is not defined

In [24]:
import gc
gc.collect()

1095